In [15]:
import cv2
import pyodbc
import os
import numpy as np
from PIL import Image
import tkinter as tk
from tkinter import messagebox, simpledialog
from tkinter import ttk

# --- GUI Setup ---
window = tk.Tk()
window.title("Face Recognition System")
window.config(background="gray")
window.geometry("550x400")

# --- Labels and Entry Fields ---
tk.Label(window, text="Name", font=("Serif", 20), bg="black", fg="white").grid(column=0, row=2, padx=10, pady=10)
name_entry = tk.Entry(window, width=25, bd=6)
name_entry.grid(column=1, row=2, padx=10, pady=10)

tk.Label(window, text="Age", font=("Serif", 20), bg="black", fg="white").grid(column=0, row=3, padx=10, pady=10)
age_entry = tk.Entry(window, width=25, bd=6)
age_entry.grid(column=1, row=3, padx=10, pady=10)

tk.Label(window, text="Place", font=("Serif", 20), bg="black", fg="white").grid(column=0, row=4, padx=10, pady=10)
place_entry = tk.Entry(window, width=25, bd=6)
place_entry.grid(column=1, row=4, padx=10, pady=10)

tk.Label(window, text="USER_ID", font=("Serif", 20), bg="black", fg="white").grid(column=0, row=5, padx=10, pady=10)
USER_ID = tk.Entry(window, width=25, bd=6)
USER_ID.grid(column=1, row=5, padx=10, pady=10)

# databse table cretaed query
# create table FACREG(
# ID INT IDENTITY(1,1) PRIMARY KEY,
# USER_ID INT,
# NAME VARCHAR(250),
# AGE VARCHAR(50),
# PLACE VARCHAR(250)
# )

# --- User Detail Button ---
def user_detail():
    name = name_entry.get()
    age = age_entry.get()
    place = place_entry.get()
    userid = USER_ID.get()


    # Check if all fields are filled
    if not name or not age or not place:
        messagebox.showwarning("Input Error", "Please fill in all details.")
        return
    
    # Print user details to the console (for debugging)
    print(f"Name: {name}, Age: {age}, Place: {place}, USER_ID:{userid}")

    # Show the info in a message box
    messagebox.showinfo("User Info", f"Name: {name}\n Age: {age}\n Place: {place}\n USER_ID:{userid}")

    # Database connection details
    try:
        conn = pyodbc.connect(
        'Driver={ODBC Driver 17 for SQL Server};'
        'Server=DESKTOP-2BCQ0PO\\SQLEXPRESS01;'  # Replace with your server name
        'Database=UMAR_DB;'  # Replace with your database name
        'UID=sa;'  # Replace with your SQL Server username
        'PWD=12345678;'  # Replace with your SQL Server password
        )
        cursor = conn.cursor()

        # Insert SQL query
        sql_query = "INSERT INTO FACREG (NAME, AGE, PLACE, USER_ID) VALUES (?, ?, ?, ?)"
        values = (name, age, place, userid)

        # Execute the query and commit the transaction
        cursor.execute(sql_query, values)
        conn.commit()

        # Show a success message
        messagebox.showinfo("Success", "User details inserted successfully.")

        # Clear the input fields after successful insertion
        name_entry.delete(0, tk.END)
        age_entry.delete(0, tk.END)
        place_entry.delete(0, tk.END)

    except Exception as e:
        messagebox.showerror("Database Error", f"Failed to insert data: {e}")
    finally:
        cursor.close()
        conn.close()

# Button to trigger user_detail() function
tk.Button(window, text="Submit User Details", bg="yellow", fg="black", padx=10, pady=10, command=user_detail).grid(column=0, row=6, padx=10, pady=10)



# --- Generate Dataset Button ---
def generate_dataset():
    user_id = simpledialog.askstring("User ID", "Enter user ID:")
    if not user_id:
        messagebox.showwarning("Input Error", "User ID is required.")
        return

    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]

    cap = cv2.VideoCapture(0)
    img_id = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        face = face_cropped(frame)
        if face is not None:
            img_id += 1
            face = cv2.resize(face, (350, 350))
            face_gray = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_dir = "data"
            os.makedirs(file_dir, exist_ok=True)
            file_path = os.path.join(file_dir, f"user.{user_id}.{img_id}.jpg")

            cv2.imwrite(file_path, face_gray)
            cv2.putText(face_gray, str(img_id), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Cropped Face", face_gray)

        if cv2.waitKey(1) == 13 or img_id == 200:  # Enter key or 50 images
            break

    cap.release()
    cv2.destroyAllWindows()
    messagebox.showinfo("Result", "Dataset generation completed.")

tk.Button(window, text="Generate Dataset", font=("Algerian", 14), bg="pink", fg="black", command=generate_dataset).grid(column=2, row=1)


# --- Train Classifier Button ---
def Train_classifier():
    data_dir = "data"
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []

    for image in path:
        img = Image.open(image).convert("L")
        image_np = np.array(img, "uint8")
        id = int(os.path.split(image)[1].split(".")[1])
        faces.append(image_np)
        ids.append(id)

    ids = np.array(ids)
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.train(faces, ids)
    clf.write("classifier.xml")
    messagebox.showinfo("Result", "Training completed and model saved.")

tk.Button(window, text="Train Classifier", font=("Algerian", 14), bg="orange", fg="red", command=Train_classifier).grid(column=0, row=1)


# --- Face Detection Button ---
# def detect_face():
#     def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
#         gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         features = classifier.detectMultiScale(gray, scaleFactor, minNeighbors)
#         for (x, y, w, h) in features:
#             cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
#             id, pred = clf.predict(gray[y:y + h, x:x + w])
#             confidence = int(100 * (1 - pred / 300))
#             name = "Unknown"

#             if confidence > 70:
#                 if id == 1:
#                     name = "Umar"
#                 elif id == 2:
#                     name = "chandan"

#             cv2.putText(img, name, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

#     def recognize(img, clf, faceCascade):
#         draw_boundary(img, faceCascade, 1.1, 10, (0, 0, 255), "Face", clf)
#         return img

#     faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
#     clf = cv2.face.LBPHFaceRecognizer_create()
#     clf.read("classifier.xml")

#     cap = cv2.VideoCapture(0)

#     while True:
#         ret, img = cap.read()
#         img = recognize(img, clf, faceCascade)
#         cv2.imshow("Face Recognition", img)
#         if cv2.waitKey(1) == 13:  # Enter key
#             break

#     cap.release()
#     cv2.destroyAllWindows()

# Function to get user name from the database based on ID
def get_name_from_db(user_id):
    try:
        conn = pyodbc.connect(
        'Driver={ODBC Driver 17 for SQL Server};'
        'Server=DESKTOP-2BCQ0PO\\SQLEXPRESS01;'  # Replace with your server name
        'Database=UMAR_DB;'  # Replace with your database name
        'UID=sa;'  # Replace with your SQL Server username
        'PWD=12345678;'  # Replace with your SQL Server password
        )
        cursor = conn.cursor()
        cursor.execute("SELECT NAME FROM FACREG WHERE ID=?", user_id)
        row = cursor.fetchone()
        return row[0] if row else "Unknown"
    except Exception as e:
        print(f"Database error: {e}")
        return "Unknown"
    finally:
        try:
            cursor.close()
            conn.close()
        except:
            pass


# Main Face Detection Function
def detect_face():
    def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = classifier.detectMultiScale(gray, scaleFactor, minNeighbors)

        for (x, y, w, h) in faces:
            face_img = gray[y:y+h, x:x+w]
            id, pred = clf.predict(face_img)
            confidence = int(100 * (1 - pred / 300))

            if confidence > 70:
                name = get_name_from_db(id)
            else:
                name = "Unknown"

            # Draw rectangle and label
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        return img

    def recognize(img, clf, faceCascade):
        return draw_boundary(img, faceCascade, 1.1, 10, (0, 255, 0), "Face", clf)

    # Load the face detection model and trained recognizer
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    clf = cv2.face.LBPHFaceRecognizer_create()
    clf.read("classifier.xml")

    # Start webcam
    cap = cv2.VideoCapture(0)
    while True:
        ret, img = cap.read()
        if not ret:
            break
        img = recognize(img, clf, faceCascade)
        cv2.imshow("Face Recognition", img)

        if cv2.waitKey(1) == 13:  # Press Enter to exit
            break

    cap.release()
    cv2.destroyAllWindows()

tk.Button(window, text="Detect Face", font=("Algerian", 14), bg="green", fg="white", command=detect_face).grid(column=1, row=1)

window.mainloop()


Name: UMAR, Age: 23, Place: Bangalore, USER_ID:1
Name: chandan, Age: 23, Place: bangalore, USER_ID:2
